Made by: Nicolas Veiga

#### Introduction
Faire Backend Exercise

Using the Faire API*1, you will write a program that accepts an API key as a single command line parameter, which should be sent with all requests in the X-FAIRE-ACCESS-TOKEN header, and does the following:

Consumes all products for a given brand*2, recording the inventory levels for each product option Consumes all orders, accepting the order if there is inventory to fulfill the order, otherwise it marks the items that don’t have enough inventory as backordered Update the inventory levels of product options as each order is moved to processing After all orders have been processed, the program should print the following metrics, plus 2 of your choosing, and then exit - your creativity is welcome! The best selling product option The largest order by dollar amount The state with the most orders

For testing, you can use the API key: HQLA9307HSLQYTC24PO2G0LITTIOHS2MJC8120PVZ83HJK4KACRZJL91QB7K01NWS2TUCFXGCHQ8HVED8WNZG0KS6XRNBFRNGY71

*1 - The base URL to use is https://www.faire-stage.com

*2 - We recommend using Tea Drops, brand token = b_d2481b88

#### Development

In [1]:
import requests
import json
import pandas as pd

API_KEY = 'HQLA9307HSLQYTC24PO2G0LITTIOHS2MJC8120PVZ83HJK4KACRZJL91QB7K01NWS2TUCFXGCHQ8HVED8WNZG0KS6XRNBFRNGY71'

In [2]:
def queryProducts(APIKey):
    products = []
    pageIndex = 1
    parameters = {"page": pageIndex}
    myUrl = "https://www.faire-stage.com/api/v1/products"
    head = {"X-FAIRE-ACCESS-TOKEN": APIKey,
            "Accept": "application/json"}
    try:
        response = requests.get(myUrl, params=parameters, headers=head)
    except:
        print("Something wrong with your internet connection, try again")
        return []

    if (response.status_code == 200):
        currentPage = response.json()["products"]
        products = currentPage

        while(len(currentPage) == response.json()["limit"]):
            pageIndex += 1
            parameters = {"page": pageIndex}
            try:
                response = requests.get(myUrl, params=parameters, headers=head)
            except:
                print("Something wrong with your internet connection, try again")
                return []

            if (response.status_code == 200):
                currentPage = response.json()["products"]
                products += currentPage
            else:
                print("Servers are down, try again")
                return []
    else:
        print("Servers are down, try again")
        return []

    return products
productsDataFrame = queryProducts(API_KEY)
print(len(productsDataFrame))

20


In [3]:
productsDataFrame = pd.DataFrame.from_dict(productsDataFrame)
productsDataFrame.head()

,active,brand_id,created_at,description,id,name,options,retail_price_cents,short_description,unit_multiplier,updated_at,wholesale_price_cents
0,False,b_d2481b88,20180423T223716.000Z,"Designed for direct-to-consumer, restaurants, ...",p_63sorm2b,Single Serve Tea Drops,"[{'id': 'po_ahmza3hm', 'created_at': '20180423...",225,"Designed for direct-to-consumer, restaurants, ...",25,20190324T080024.000Z,90
1,True,b_d2481b88,20180423T223717.000Z,Compostable Box: (10 Drops/Box) Compostable Bo...,p_c2dny7bu,Tea Drops in Compostable Box,"[{'id': 'po_etc229aa', 'created_at': '20180423...",1000,Compostable Box: (10 Drops/Box) Compostable Bo...,6,20190324T080024.000Z,585
2,True,b_d2481b88,20180423T223717.000Z,8 drops. This package is ideal for retailers j...,p_9d0tbjep,Tea Drops in Medium Wood Box,"[{'id': 'po_6rjvcsjc', 'created_at': '20180423...",1400,8 drops. This package is ideal for retailers j...,1,20190324T080024.000Z,675
3,True,b_d2481b88,20180423T223717.000Z,The must-have gift for any tea enthusiast. Our...,p_4y2z6ra2,Large Sampler Box,"[{'id': 'po_5i52cny4', 'created_at': '20180423...",3400,The must-have gift for any tea enthusiast. Our...,1,20190324T080024.000Z,1700
4,False,b_d2481b88,20180423T223717.000Z,Fits 8 Tea Drops,p_51poeros,Unfilled Medium Wood Box,"[{'id': 'po_cdm9klqb', 'created_at': '20180423...",500,Fits 8 Tea Drops,1,20190324T080024.000Z,280


Using pandas to handle the tables

In [4]:
dataframeOptions = pd.DataFrame.from_dict(productsDataFrame["options"][0])
dataframeOptions

,active,available_quantity,created_at,id,name,product_id,sku,updated_at
0,True,0,20180423T223716.000Z,po_ahmza3hm,Rose Earl Grey,p_63sorm2b,SGS0001,20190324T034954.000Z
1,True,0,20180423T223716.000Z,po_8dc5r8ws,Citrus Ginger,p_63sorm2b,SGS0002,20190323T223212.000Z
2,True,0,20180423T223716.000Z,po_c9xzsf2x,Sweet Peppermint,p_63sorm2b,SGS0003,20190323T131429.000Z


In [5]:
# Grouping the options so we can have everythin in one table
dataframeOptions = pd.DataFrame.from_dict(productsDataFrame["options"][0])
for i in range(1, len(productsDataFrame)):
    dataframeOptions = dataframeOptions.append(pd.DataFrame.from_dict(productsDataFrame["options"][i]))

C:\Users\veiga\AppData\Local\conda\conda\envs\py3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [6]:
dataframeOptions.head()

,active,available_quantity,created_at,id,name,product_id,sku,updated_at
0,True,0.0,20180423T223716.000Z,po_ahmza3hm,Rose Earl Grey,p_63sorm2b,SGS0001,20190324T034954.000Z
1,True,0.0,20180423T223716.000Z,po_8dc5r8ws,Citrus Ginger,p_63sorm2b,SGS0002,20190323T223212.000Z
2,True,0.0,20180423T223716.000Z,po_c9xzsf2x,Sweet Peppermint,p_63sorm2b,SGS0003,20190323T131429.000Z
0,False,-30.0,20180423T223717.000Z,po_etc229aa,Rose Earl Grey,p_c2dny7bu,COMPF001,20190324T034954.000Z
1,True,-30.0,20180423T223717.000Z,po_b2v6owmt,Citrus Ginger,p_c2dny7bu,COMPF002,20190323T223227.000Z


In [7]:
inventory = dataframeOptions[(dataframeOptions.available_quantity.notnull()) & (dataframeOptions.available_quantity > 0)]
inventory

,active,available_quantity,created_at,id,name,product_id,sku,updated_at
0,True,434.0,20180423T223717.000Z,po_5i52cny4,Rose Earl Grey,p_4y2z6ra2,WOODF001,20190323T211532.000Z
0,False,10.0,20180423T223717.000Z,po_b2ksg5wi,Rose Early Grey,p_4kneqjyb,IND0001,20190323T231307.000Z
0,False,10.0,20180423T223717.000Z,po_4bj8rwu9,default,p_7pid8vps,WOODLARGE,20190324T122704.000Z
0,False,434.0,20190204T222051.000Z,po_em65etwg73,Rose Earl Grey,p_2k5o128gwg,WOODF001,20190324T122704.000Z


In [8]:
def queryOrders(API_KEY):
    orders = []
    pageIndex = 1
    parameters = {"page": pageIndex}
    myUrl = "https://www.faire-stage.com/api/v1/orders"
    head = {"X-FAIRE-ACCESS-TOKEN": API_KEY,
            "Accept": "application/json"}
    try:
        response = requests.get(myUrl, params=parameters, headers=head)
    except:
        print("Something wrong with your internet connection, try again")
        return []

    if (response.status_code == 200):
        currentPage = response.json()["orders"]
        orders = currentPage
        
        while(len(currentPage) == response.json()["limit"]):
            pageIndex += 1
            parameters = {"page": pageIndex}
            try:
                response = requests.get(myUrl, params=parameters, headers=head)
            except:
                print("Something wrong with your internet connection, try again")
                return []
            if (response.status_code == 200):
                currentPage = response.json()["orders"]
                orders += currentPage
            else:
                print("Something wrong with servers")
                return [] 
    else:
        print("Something wrong with servers")
        return []

    return orders
orders = queryOrders(API_KEY)
print(len(orders))

93


In [9]:
ordersDataFrame = pd.DataFrame.from_dict(orders)
ordersDataFrame.head()

,address,created_at,id,items,ship_after,shipments,state,updated_at
0,"{'name': 'luan', 'address1': 'wejfalf', 'addre...",20180510T181535.000Z,bo_3s7ei5an,"[{'id': 'oi_6x9ejf4p', 'created_at': '20180510...",20180510T181535.000Z,[],CANCELED,20181031T192456.000Z
1,"{'name': 'kllkjlkjlkj', 'address1': 'ljkljkljl...",20181025T202332.000Z,bo_3iuhowot,"[{'id': 'oi_ce3i14rx', 'created_at': '20181025...",20181025T202332.000Z,"[{'id': 's_a7nrmvu4', 'created_at': '20181025T...",DELIVERED,20181108T210001.000Z
2,"{'name': 'asfaf', 'address1': 'asfas', 'postal...",20181026T134537.000Z,bo_3i2mfdbj,"[{'id': 'oi_7vi9dze7', 'created_at': '20181026...",20181026T134537.000Z,"[{'id': 's_2osshc9f', 'created_at': '20181026T...",DELIVERED,20181109T140000.000Z
3,"{'name': 'asfasa', 'address1': 'asfasf', 'post...",20181026T141458.000Z,bo_2fhfjja9,"[{'id': 'oi_3oocl7pb', 'created_at': '20181026...",20181026T141458.000Z,"[{'id': 's_9livqhn6', 'created_at': '20181026T...",DELIVERED,20181109T150000.000Z
4,"{'name': 'Guest Store', 'address1': '1455 Nort...",20181029T172532.000Z,bo_ei8yw53s,"[{'id': 'oi_6w08p2qr', 'created_at': '20181029...",20181029T172532.000Z,"[{'id': 's_7wa62cnd', 'created_at': '20181029T...",DELIVERED,20181112T180001.000Z


In [10]:
ordersDataFrame.state.value_counts()

PROCESSING    48
DELIVERED     41
CANCELED       3
IN_TRANSIT     1
Name: state, dtype: int64

Have to update the itens that are in PROCESSING state. If there is inventory, we subtract the amount of itens from inventory and keep the state as PROCESSING. If there is no inventory we change the state to BACKORDERED.
I need to pay attention to shipt_after and created_at paramenters. Since there is no way from the data to know how long it takes for a order to be ready, I will considere shipt_after as my variable that controls with order is processed first.

Leave only PROCESSING values in table and sort them by ship_after:

In [11]:
ordersInProcessingState = ordersDataFrame[ordersDataFrame.state == "PROCESSING"]
ordersInProcessingState = ordersInProcessingState.sort_values(by=['ship_after'])
ordersInProcessingState.head()

,address,created_at,id,items,ship_after,shipments,state,updated_at
90,"{'name': 'paul test', 'address1': '1230 U.S. 1...",20181218T141258.000Z,bo_alanim4r,"[{'id': 'oi_blzj70us', 'created_at': '20181218...",20181218T141258.000Z,[],PROCESSING,20190324T131209.000Z
91,"{'name': 'paul test', 'address1': '3002 Broadw...",20181219T014333.000Z,bo_2ppdbqvn,"[{'id': 'oi_dcaed3su', 'created_at': '20181219...",20181219T014333.000Z,[],PROCESSING,20190324T131209.000Z
87,"{'name': 'Tw', 'address1': 'Fe', 'address2': '...",20181219T014520.000Z,bo_2x2y799i,"[{'id': 'oi_g55irsr1', 'created_at': '20181219...",20181219T014520.000Z,[],PROCESSING,20190324T131208.000Z
56,"{'name': 'Marcelo Roles', 'address1': '300 Cal...",20181219T161843.000Z,bo_bg4cwfos,"[{'id': 'oi_7nx8hl22', 'created_at': '20181219...",20181219T161843.000Z,[],PROCESSING,20190324T131200.000Z
60,"{'name': 'Olivia Chen', 'address1': '525 2nd S...",20181220T005231.000Z,bo_746s4oh4,"[{'id': 'oi_75arwwnw', 'created_at': '20181220...",20181220T005231.000Z,[],PROCESSING,20190324T131201.000Z


Now we have to iterate over all the orders and evaluate if the items are available. If the items are available, change the status to processing. Otherwihse, change the status to BACKORDERED.

In [12]:
def updateState(order, ableToFullfill, index):
    if(not ableToFullfill):
        #TODO: should we be sending Backordering Items, sending a post command
        #https://www.faire.com/api/v1/orders/<ID>/items/availability
        index = ordersDataFrame[ordersDataFrame.id == orderIds.values[index]].index[0]
        ordersDataFrame.state.iloc[index] = "BACKORDERED"

I am not considering that we can have an order, with multiple items of the same product (product id and product option id). If that is allowed, I will have to verify every item of an order and accumulate the amount of products, and them compare with the amount in inventory.

In [13]:
def updateInventory(order):
    for item in order:
        #TODO: send a PATCH https://www.faire.com/api/v1/products/options/<ID>
        optionsDataFrame = pd.DataFrame.from_dict(productsDataFrame.options[productsDataFrame.id == item["product_id"]].values[0])
        index = optionsDataFrame[optionsDataFrame.id == item["product_option_id"]].index[0]
        optionsDataFrame.available_quantity.iloc[index] -= item["quantity"]
        
        index = productsDataFrame.options[productsDataFrame.id == item["product_id"]].index[0]
        productsDataFrame.options.iloc[index] = optionsDataFrame.to_dict("records")

In [14]:
orderIds = ordersInProcessingState["id"]
for index, order in enumerate(ordersInProcessingState["items"]):
    ableToFullfill = False
    for item in order:
        if(not inventory[(inventory["product_id"] == item["product_id"]) & 
                         (inventory["id"] == item["product_option_id"])].empty):
            if (item["quantity"] <= 
                inventory[(inventory["product_id"] == item["product_id"]) & 
                          (inventory["id"] == item["product_option_id"])].available_quantity.values[0]):
                ableToFullfill = True
            else:
                ableToFullfill = False
                break
        else:
            ableToFullfill = False
            break
    if(ableToFullfill):
        updateInventory(order)
    updateState(order, ableToFullfill, index)

In [15]:
ordersDataFrame

,address,created_at,id,items,ship_after,shipments,state,updated_at
0,"{'name': 'luan', 'address1': 'wejfalf', 'addre...",20180510T181535.000Z,bo_3s7ei5an,"[{'id': 'oi_6x9ejf4p', 'created_at': '20180510...",20180510T181535.000Z,[],CANCELED,20181031T192456.000Z
1,"{'name': 'kllkjlkjlkj', 'address1': 'ljkljkljl...",20181025T202332.000Z,bo_3iuhowot,"[{'id': 'oi_ce3i14rx', 'created_at': '20181025...",20181025T202332.000Z,"[{'id': 's_a7nrmvu4', 'created_at': '20181025T...",DELIVERED,20181108T210001.000Z
2,"{'name': 'asfaf', 'address1': 'asfas', 'postal...",20181026T134537.000Z,bo_3i2mfdbj,"[{'id': 'oi_7vi9dze7', 'created_at': '20181026...",20181026T134537.000Z,"[{'id': 's_2osshc9f', 'created_at': '20181026T...",DELIVERED,20181109T140000.000Z
3,"{'name': 'asfasa', 'address1': 'asfasf', 'post...",20181026T141458.000Z,bo_2fhfjja9,"[{'id': 'oi_3oocl7pb', 'created_at': '20181026...",20181026T141458.000Z,"[{'id': 's_9livqhn6', 'created_at': '20181026T...",DELIVERED,20181109T150000.000Z
4,"{'name': 'Guest Store', 'address1': '1455 Nort...",20181029T172532.000Z,bo_ei8yw53s,"[{'id': 'oi_6w08p2qr', 'created_at': '20181029...",20181029T172532.000Z,"[{'id': 's_7wa62cnd', 'created_at': '20181029T...",DELIVERED,20181112T180001.000Z
5,"{'name': 'Jeffs Place', 'address1': '888 7th S...",20181030T185735.000Z,bo_nlq8e8rl,"[{'id': 'oi_4z561rve', 'created_at': '20181030...",20181030T185735.000Z,"[{'id': 's_4dzjt809', 'created_at': '20181030T...",DELIVERED,20181113T190001.000Z
6,"{'name': 'Jeffrey Gulbronson', 'address1': '36...",20181031T190535.000Z,bo_39u2cikq,"[{'id': 'oi_1yyokxje', 'created_at': '20181031...",20181031T190535.000Z,"[{'id': 's_bztvxhz6', 'created_at': '20181031T...",DELIVERED,20181114T200001.000Z
7,"{'name': 'kljlkj', 'address1': 'jlkjkl', 'post...",20181031T210839.000Z,bo_ee2s5izt,"[{'id': 'oi_ekwyje54', 'created_at': '20181031...",20181031T210839.000Z,"[{'id': 's_81dpomdc', 'created_at': '20181031T...",DELIVERED,20181114T220002.000Z
8,"{'name': 'asd', 'address1': '55 Ivan Allen Jun...",20181031T211033.000Z,bo_fst2hhzi,"[{'id': 'oi_bmss68y6', 'created_at': '20181031...",20181031T211033.000Z,"[{'id': 's_96kejbtw', 'created_at': '20181031T...",DELIVERED,20181114T220002.000Z
9,"{'name': 'dsfasfasf', 'address1': 'asfasfas', ...",20181025T194243.000Z,bo_6s8tvwdg,"[{'id': 'oi_4ijzwzgz', 'created_at': '20181025...",20181025T194243.000Z,"[{'id': 's_6uxedu1z', 'created_at': '20181025T...",DELIVERED,20181119T160002.000Z


In [16]:
ordersDataFrame.state.value_counts()

BACKORDERED    48
DELIVERED      41
CANCELED        3
IN_TRANSIT      1
Name: state, dtype: int64

#### Queries
##### 1 - The best selling product option

Group by product id and product option id and sum the output.

In [17]:
def getBestSellingProductOption(ordersDataFrame):
    #Flat items list from ordersDataFrame
    itemsDataFrame = pd.DataFrame.from_dict([item for sublist in ordersDataFrame["items"].tolist() for item in sublist])
    #Group by product_id and product_option_id summing quantity and return a dataframe sorted in non ascending order
    bestSelling = itemsDataFrame.groupby(['product_id' ,'product_option_id']).sum().sort_values(by=['quantity'], ascending = False)
    return bestSelling.index[0][1]

getBestSellingProductOption(ordersDataFrame)

'po_bm2czlhz'

##### 2 - The largest order by dollar amount

In [18]:
def getLargestOrder(ordersDataFrame):
    #Flat items list from ordersDataFrame
    itemsDataFrame = pd.DataFrame.from_dict([item for sublist in ordersDataFrame["items"].tolist() for item in sublist])
    groupbyOrderId = itemsDataFrame.groupby(['order_id']).sum()
    orderValue = groupbyOrderId["price_cents"] * groupbyOrderId["quantity"]
    groupbyOrderId["total_price"] = orderValue
    largestOrderDataFrame = groupbyOrderId.sort_values(by=['total_price'], ascending = False).head()
    return largestOrderDataFrame.index[0]

getLargestOrder(ordersDataFrame)

'bo_6a0lzo8kr1'

##### 3 - The state with the most orders

In [19]:
def getStateWithMostOrders(ordersDataFrame):
    addressDataFrame = pd.DataFrame.from_dict(ordersDataFrame["address"].tolist())
    return addressDataFrame.state.value_counts().index[0]

getStateWithMostOrders(ordersDataFrame)

'California'

### Final code

In [21]:
import requests
import json
import pandas as pd

API_KEY = 'HQLA9307HSLQYTC24PO2G0LITTIOHS2MJC8120PVZ83HJK4KACRZJL91QB7K01NWS2TUCFXGCHQ8HVED8WNZG0KS6XRNBFRNGY71'

def queryProducts(APIKey):
    products = []
    pageIndex = 1
    parameters = {"page": pageIndex}
    myUrl = "https://www.faire-stage.com/api/v1/products"
    head = {"X-FAIRE-ACCESS-TOKEN": APIKey,
            "Accept": "application/json"}
    try:
        response = requests.get(myUrl, params=parameters, headers=head)
    except:
        print("Something wrong with your internet connection, try again")
        return []

    if (response.status_code == 200):
        currentPage = response.json()["products"]
        products = currentPage

        while(len(currentPage) == response.json()["limit"]):
            pageIndex += 1
            parameters = {"page": pageIndex}
            try:
                response = requests.get(myUrl, params=parameters, headers=head)
            except:
                print("Something wrong with your internet connection, try again")
                return []

            if (response.status_code == 200):
                currentPage = response.json()["products"]
                products += currentPage
            else:
                print("Servers are down, try again")
                return []
    else:
        print("Servers are down, try again")
        return []

    return products

def queryOrders(API_KEY):
    orders = []
    pageIndex = 1
    parameters = {"page": pageIndex}
    myUrl = "https://www.faire-stage.com/api/v1/orders"
    head = {"X-FAIRE-ACCESS-TOKEN": API_KEY,
            "Accept": "application/json"}
    try:
        response = requests.get(myUrl, params=parameters, headers=head)
    except:
        print("Something wrong with your internet connection, try again")
        return []

    if (response.status_code == 200):
        currentPage = response.json()["orders"]
        orders = currentPage
        
        while(len(currentPage) == response.json()["limit"]):
            pageIndex += 1
            parameters = {"page": pageIndex}
            try:
                response = requests.get(myUrl, params=parameters, headers=head)
            except:
                print("Something wrong with your internet connection, try again")
                return []
            if (response.status_code == 200):
                currentPage = response.json()["orders"]
                orders += currentPage
            else:
                print("Something wrong with servers")
                return [] 
    else:
        print("Something wrong with servers")
        return []

    return orders

def updateState(ordersDataFrame, ableToFullfill, index, orderIds):
    if(not ableToFullfill):
        #TODO: should we be sending Backordering Items, sending a post command
        #https://www.faire.com/api/v1/orders/<ID>/items/availability
        index = ordersDataFrame[ordersDataFrame.id == orderIds.values[index]].index[0]
        ordersDataFrame.state.iloc[index] = "BACKORDERED"

def updateInventory(order, productsDataFrame):
    for item in order:
        #TODO: send a PATCH https://www.faire.com/api/v1/products/options/<ID>
        optionsDataFrame = pd.DataFrame.from_dict(productsDataFrame.options[productsDataFrame.id == item["product_id"]].values[0])
        index = optionsDataFrame[optionsDataFrame.id == item["product_option_id"]].index[0]
        optionsDataFrame.available_quantity.iloc[index] -= item["quantity"]
        
        index = productsDataFrame.options[productsDataFrame.id == item["product_id"]].index[0]
        productsDataFrame.options.iloc[index] = optionsDataFrame.to_dict("records")
        
def processOrders(ordersInProcessingState, inventory, ordersDataFrame, productsDataFrame):
    orderIds = ordersInProcessingState["id"]
    for index, order in enumerate(ordersInProcessingState["items"]):
        ableToFullfill = False
        for item in order:
            if(not inventory[(inventory["product_id"] == item["product_id"]) & 
                             (inventory["id"] == item["product_option_id"])].empty):
                if (item["quantity"] <= 
                    inventory[(inventory["product_id"] == item["product_id"]) & 
                              (inventory["id"] == item["product_option_id"])].available_quantity.values[0]):
                    ableToFullfill = True
                else:
                    ableToFullfill = False
                    break
            else:
                ableToFullfill = False
                break
        if(ableToFullfill):
            updateInventory(order, productsDataFrame)
        updateState(ordersDataFrame, ableToFullfill, index, orderIds)
        
def getBestSellingProductOption(ordersDataFrame):
    #Flat items list from ordersDataFrame
    itemsDataFrame = pd.DataFrame.from_dict([item for sublist in ordersDataFrame["items"].tolist() for item in sublist])
    #Group by product_id and product_option_id summing quantity and return a dataframe sorted in non ascending order
    bestSelling = itemsDataFrame.groupby(['product_id' ,'product_option_id']).sum().sort_values(by=['quantity'], ascending = False)
    return bestSelling.index[0][1]

def getLargestOrder(ordersDataFrame):
    #Flat items list from ordersDataFrame
    itemsDataFrame = pd.DataFrame.from_dict([item for sublist in ordersDataFrame["items"].tolist() for item in sublist])
    groupbyOrderId = itemsDataFrame.groupby(['order_id']).sum()
    orderValue = groupbyOrderId["price_cents"] * groupbyOrderId["quantity"]
    groupbyOrderId["total_price"] = orderValue
    largestOrderDataFrame = groupbyOrderId.sort_values(by=['total_price'], ascending = False).head()
    return largestOrderDataFrame.index[0]

def getStateWithMostOrders(ordersDataFrame):
    addressDataFrame = pd.DataFrame.from_dict(ordersDataFrame["address"].tolist())
    return addressDataFrame.state.value_counts().index[0]

def execute(API_KEY):
    productsDataFrame = queryProducts(API_KEY)
    if(len(productsDataFrame) > 0):
        productsDataFrame = pd.DataFrame.from_dict(productsDataFrame)
        
        # Grouping the options so we can have everythin in one table
        optionsDataFrame = pd.DataFrame.from_dict([item for sublist in productsDataFrame["options"].tolist() for item in sublist])
        
        inventory = optionsDataFrame[(optionsDataFrame.available_quantity.notnull()) & (optionsDataFrame.available_quantity > 0)]

        orders = queryOrders(API_KEY)
        if(len(orders) > 0):
            ordersDataFrame = pd.DataFrame.from_dict(orders)
            
            ordersSorteredProcessingState = ordersDataFrame[ordersDataFrame.state == "PROCESSING"]
            ordersSorteredProcessingState = ordersSorteredProcessingState.sort_values(by=['ship_after'])
            
            processOrders(ordersSorteredProcessingState, inventory, ordersDataFrame, productsDataFrame)
            
            #Queries
            print(getBestSellingProductOption(ordersDataFrame))
            print(getLargestOrder(ordersDataFrame))
            print(getStateWithMostOrders(ordersDataFrame))
        else:
            print("No Orders to process")
    else:
        print("No Products to process")

execute(API_KEY)

po_bm2czlhz
bo_6a0lzo8kr1
California
